In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import datetime
import warnings
import random
from functools import reduce
from os import walk
warnings.filterwarnings('ignore')

In [3]:
datas = pd.read_csv('F:/aliyuun_datas/2021数字汽车大赛/temp_df/datasets_aw.csv')
datas.shape

(6880557, 44)

In [5]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880557 entries, 0 to 6880556
Data columns (total 44 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   vin                            object 
 1   时间                             object 
 2   车速                             float64
 3   车辆状态                           float64
 4   充电状态                           float64
 5   总电压                            float64
 6   总电流                            float64
 7   累计里程                           float64
 8   SOC                            float64
 9   电池单体电压最高值                      float64
 10  最高电压电池单体代号                     float64
 11  电池单体电压最低值                      float64
 12  最低电压电池单体代号                     float64
 13  最高温度值                          float64
 14  最高温度探针单体代号                     float64
 15  最低温度值                          float64
 16  最低温度探针单体代号                     float64
 17  最高报警等级                         float64
 18  DC

In [6]:
datas[['经度_', '维度_', '经度', '维度']]

,经度_,维度_,经度,维度
0,116.359166,39.839881,116.359166,39.839881
1,116.359166,39.839881,116.359166,39.839881
2,116.359166,39.839881,116.359166,39.839881
3,116.359166,39.839881,116.359166,39.839881
4,116.359166,39.839881,116.359166,39.839881
...,...,...,...,...
6880552,116.359218,39.840158,116.359218,39.840158
6880553,116.359218,39.840158,116.359218,39.840158
6880554,116.359218,39.840158,116.359218,39.840158
6880555,116.359218,39.840158,116.359218,39.840158


# <font color=red>构造车辆没15秒的移动距离特征 **distance** 和转角特征 **degree** </font>

In [7]:
"""计算直线距离函数"""
def getDistance(latA, lonA, latB, lonB):
    ra = 6378140  # 赤道半径
    rb = 6356755  # 极半径
    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)

    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB)) ** 2 / math.cos(x / 2) ** 2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB)) ** 2 / math.sin(x / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    distance = round(distance / 1000, 4)
    return distance*1000

In [ ]:
def getDistance(latA, lonA, latB, lonB):
    ra = 6378140    # 赤道半径
    rb = 6356755    # 极半径
    flatten = (ra - rb) / ra
    # 四列特征弧度转换
    radLatA = latA.apply(lambda radlat: math.radians(radlat))
    radLonA = lonB.apply(lambda radlat: math.radians(radlat))
    radLatB = latB.apply(lambda radlat: math.radians(radlat))
    radLonB = lonB.apply(lambda radlat: math.radians(radlat))
    temp = radLatA - radLonB
    print(radLatA)
    pA = radLatA.apply(lambda rltA: rb/ra*math.tan(rltA))
    pB = radLatB.apply(lambda rltB: rb/ra*math.tan(rltB))
    x = pA.apply(lambda x1: math.sin(x1)) * pB.apply(lambda x2: math.sin(x2)) + pA.apply(lambda x3: math.cos(x3)) * pB.apply(lambda x4: math.cos(x4) * temp.apply(lambda x5: math.cos(x5)))
    x = x.apply(lambda x6: math.acos(x6))
    c1 = x.apply(lambda y1: math.sin(x)) - x
    c1 = c1 * pA.apply(lambda y2: math.sin(y2))
 
getDistance(data01['经度_'], data01['维度_'], data01['经度'], data01['维度'])

In [8]:
"""计算角度函数"""
def getDegree(latA, lonA, latB, lonB):
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
    dLon = radLonB - radLonA
    y = math.sin(dLon) * math.cos(radLatB)
    x = math.cos(radLatA) * math.sin(radLatB) - math.sin(radLatA) * math.cos(radLatB) * math.cos(dLon)
    brng = math.degrees(math.atan2(y, x))
    brng = round((brng + 360) % 360, 4)
    brng = int(brng)
    if (brng == 0.0) or ((brng == 360.0)):
        return 90
    elif brng == 90.0:
        return 0
    elif brng == 180.0:
        return 270
    elif brng == 270.0:
        return 180
    elif 0 < brng < 90:
        return 90-brng
    elif 90 < brng < 180:
        return 360-brng
    elif 180 < brng < 270:
        return 180+brng
    elif 270 < brng < 360:
        return 90+brng
    else:
        pass

In [37]:
def init_angle(latA, lonA, latB, lonB):
    radLatA = np.array(latA.apply(lambda radlat: math.radians(radlat)))
    radLonA = np.array(lonB.apply(lambda radlat: math.radians(radlat)))
    radLatB = np.array(latB.apply(lambda radlat: math.radians(radlat)))
    radLonB = np.array(lonB.apply(lambda radlat: math.radians(radlat)))
    dLon = radLonB - radLonA
    datas['radLatA'] = radLatA
    datas['radLonA'] = radLonA
    datas['radLatB'] = radLatB
    datas['radLonB'] = radLonB
    datas['dLon'] = dLon
    # y = math.sin(dLon) * math.cos(radLatB)
    # x = math.cos(radLatA) * math.sin(radLatB) - math.sin(radLatA) * math.cos(radLatB) * math.cos(dLon)
    # brng = math.degrees(math.atan2(y, x))
    # brng = round((brng + 360) % 360, 4)
    # brng = int(brng)
    # y = datas['dLon'].apply(lambda dL: math.sin(dL) * datas['radLatB'].apply(lambda rltB: math.cos(rltB)))
    datas['y1'] = datas['dLon'].apply(lambda dL: math.sin(dL))
    datas['y2'] = datas['radLatB'].apply(lambda rltB: math.cos(rltB))
    datas['y'] = datas['y1'] * datas['y2']
    datas['x1'] = datas['radLatA'].apply(lambda rltA: math.cos(rltA))
    datas['x2'] = datas['radLatB'].apply(lambda rltB: math.sin(rltB))
    datas['x3'] = datas['radLatA'].apply(lambda rltA: math.sin(rltA))
    datas['x4'] = datas['radLatB'].apply(lambda rltB: math.cos(rltB))
    datas['x5'] = datas['dLon'].apply(lambda dL: math.cos(dL))
    datas['x'] = datas['x1'] * datas['x2'] - datas['x3'] * datas['x4'] * datas['x5']
    


init_angle(datas['经度_'], datas['维度_'], datas['经度'], datas['维度'])

In [33]:
datas[['radLatA', 'radLonA', 'radLatB', 'radLonB', 'dLon', 'y1', 'y2', 'y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x', 'brng']]

,radLatA,radLonA,radLatB,radLonB,dLon,y1,y2,y,x1,x2,x3,x4,x5,x
0,2.030851,0.695337,2.030851,0.695337,0.0,0.0,-0.443997,-0.0,-0.443997,0.896028,0.896028,-0.443997,1.0,0.0
1,2.030851,0.695337,2.030851,0.695337,0.0,0.0,-0.443997,-0.0,-0.443997,0.896028,0.896028,-0.443997,1.0,0.0
2,2.030851,0.695337,2.030851,0.695337,0.0,0.0,-0.443997,-0.0,-0.443997,0.896028,0.896028,-0.443997,1.0,0.0
3,2.030851,0.695337,2.030851,0.695337,0.0,0.0,-0.443997,-0.0,-0.443997,0.896028,0.896028,-0.443997,1.0,0.0
4,2.030851,0.695337,2.030851,0.695337,0.0,0.0,-0.443997,-0.0,-0.443997,0.896028,0.896028,-0.443997,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6880552,2.030851,0.695342,2.030851,0.695342,0.0,0.0,-0.443998,-0.0,-0.443998,0.896028,0.896028,-0.443998,1.0,0.0
6880553,2.030851,0.695342,2.030851,0.695342,0.0,0.0,-0.443998,-0.0,-0.443998,0.896028,0.896028,-0.443998,1.0,0.0
6880554,2.030851,0.695342,2.030851,0.695342,0.0,0.0,-0.443998,-0.0,-0.443998,0.896028,0.896028,-0.443998,1.0,0.0
6880555,2.030851,0.695342,2.030851,0.695342,0.0,0.0,-0.443998,-0.0,-0.443998,0.896028,0.896028,-0.443998,1.0,0.0


In [34]:
datas['x'].value_counts()

 0.000000e+00    2477055
 8.726646e-08       8043
-8.726646e-08       7682
 3.490659e-08       6286
-3.490659e-08       6190
                  ...   
 8.059057e-04          1
-4.039390e-04          1
-1.024316e-03          1
 1.333432e-05          1
-1.248783e-04          1
Name: x, Length: 198556, dtype: int64